In [41]:
import numpy as np
import pickle

from scipy.sparse import rand
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix

import implementations as imp
import proj1_helpers as ph

In [42]:
glove = np.load('embeddings.npy')
y = pickle.load(open('cooc.pkl', 'rb'))
vocab = pickle.load(open('vocab.pkl', 'rb'))

In [43]:
glove.shape

(21161, 20)

In [44]:
print(glove[1])

[ 0.56403988 -0.33822548  0.11260476  0.05975893 -0.39520949 -0.2375117
 -0.12390807 -0.16872797 -0.02044019  0.32865794 -0.64644995 -0.08901839
  0.26161983 -0.1631089   0.08588484  0.57699542 -0.52317936 -0.37408539
 -0.13934747 -0.61597906]


In [45]:
thibault = list(zip(y.row, y.col, y.data))

In [46]:
inv_vocab = {v: k for k, v in vocab.items()}

In [47]:
print(list(map(lambda t : [inv_vocab[t[0]], inv_vocab[t[1]], t[2]], thibault[:100])))

[['<user>', '<user>', 207302], ['!', '<user>', 73069], ['i', '<user>', 57298], ['the', '<user>', 30095], ['.', '<user>', 37914], [',', '<user>', 31220], ['to', '<user>', 34338], ['you', '<user>', 42215], ['(', '<user>', 8646], ['<url>', '<user>', 7980], ['a', '<user>', 23651], ['...', '<user>', 10669], ['and', '<user>', 21441], ['my', '<user>', 18235], ['me', '<user>', 20487], ['of', '<user>', 11016], ['?', '<user>', 24963], ['is', '<user>', 12825], ['for', '<user>', 14617], ['in', '<user>', 12590], ['it', '<user>', 17626], ['"', '<user>', 16643], ['this', '<user>', 7707], ['so', '<user>', 11216], ['-', '<user>', 5440], ['with', '<user>', 8600], ['on', '<user>', 10289], ['that', '<user>', 11599], [')', '<user>', 8376], ['be', '<user>', 9940], ["i'm", '<user>', 10328], ['have', '<user>', 10408], [':', '<user>', 4367], ['but', '<user>', 9315], ['just', '<user>', 8257], ['rt', '<user>', 19209], ['love', '<user>', 7683], ['your', '<user>', 8459], ['all', '<user>', 5858], ['not', '<user>', 

In [48]:
def split_id_tweet (list_tweets) :
    ids = [x.split(',')[0] for x in list_tweets]
    tweets =  [','.join(x.split(',')[:1]) for x in list_tweets]
    return ids , tweets

In [49]:
tweets = []
smile = []
with open('twitter-datasets/train_neg_full.txt', encoding='UTF-8') as f :
    ids , read = split_id_tweet(f.readlines())
    smile = [-1] * len(read)
    tweets_train = read
    
with open('twitter-datasets/train_pos_full.txt', encoding='UTF-8') as f :
    ids , read = split_id_tweet(f.readlines())
    smile += [1] * len(read)
    tweets_train += read
    


In [50]:
len(tweets_train)


2500000

In [51]:
size_vector = len(vocab)
def to_vector (line) :
    res= [0] * size_vector
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res[change_nb] = 1
    return res
        
#tweets_res = [ np.array(to_vector(line)).dot(glove) for line in tweets]


In [52]:
def to_three (idx , line ) :
    res_set= []
    res = []
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res_set += [change_nb]
    for change_nb in set(res_set) :
        res += [[1 , idx , change_nb]]
    return res
#tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three(idx , line)]

def columns ( matrix , i) :
    return [x[i] for x in matrix]


def tweet_to_matrix ( tweets ) :
    tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three(idx , line)]
    
    sparse_tweets = coo_matrix((columns(tweets_to_sparse,0) , (columns(tweets_to_sparse,1),columns(tweets_to_sparse,2))) , shape=(len(tweets), len(glove)))
    return sparse_tweets.dot(glove)

In [53]:
#sparse_tweets = coo_matrix((columns(0) , (columns(1),columns(2))) , shape=(len(tweets), len(glove)))
#tweets_res = sparse_tweets.dot(glove)
tweets_res = tweet_to_matrix(tweets_train)


In [62]:
count = 0
count2 = 0
for x in tweets_res :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res) , count/len(tweets_res) , count2

(5989, 2500000, 0.0023956, 2500000)

In [55]:
w,loss = imp.least_squares(np.array(smile), tweets_res)

In [56]:
w

array([-0.00497053, -0.00257374,  0.03705977, -0.00192451, -0.03340073,
       -0.02537296,  0.00685164, -0.01524033,  0.00786395, -0.01178066,
        0.00888847, -0.00923736, -0.01688849, -0.03289105, -0.00933731,
       -0.0018966 , -0.00220792,  0.01008644,  0.01424898,  0.00859098])

In [57]:
ids_test=[]
tweets_test=[]
with open('twitter-datasets/test_data.txt') as f :
    ids , tweets_test = split_id_tweet(f.readlines())
    
tweets_res_test = tweet_to_matrix(tweets_test)


In [63]:
count = 0
count2 = 0
for x in tweets_res_test :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res_test) , count/len(tweets_res_test) , count2

(9608, 10000, 0.9608, 10000)

In [58]:
res = tweets_res_test.dot(w)
res = np.sign(res)

In [59]:
imp.create_csv_submission(ids,res,'results.csv')

In [60]:
count = 0
for x in res :
    if x  == 0 :
        count +=1
count , len(res) , count/len(res)

(9608, 10000, 0.9608)